In [5]:

import warnings
warnings.filterwarnings("ignore")

In [6]:
from environments.live_environments import BaseLiveTradingEnv
from neuralforecast.core import NeuralForecast
from configs import defaults
from Keys import *
import pickle
import numpy as np
from utils import pearl_utils
from Pearl.pearl.utils.instantiations.environments.gym_environment import \
    GymEnvironment
import boto3

In [7]:

conn = boto3.resource('s3')
AWS_ACCESS_KEY
AWS_SECRET_KEY
s3= boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)
s3.download_file('coinbasetradehistory','trade.db','Trade_history/trade.db')
# conn.download_file(
#     # Filename='https://coinbasetradehistory.s3.us-east-1.amazonaws.com/trade.db'
# )
s3.download_file('coinbasetradehistory','trade.db','Trade_history/trade.db')

In [8]:
forecast_model=NeuralForecast.load('MultiHeadForecastingModel/')

Seed set to 6
Seed set to 19
Seed set to 16


In [9]:
base_asset='DOGE'
quote_asset='USDC'
test_net=False
time_frame='1h'
# product_type='FUTURE'
product_type='Spot'
futures_target='DOG-29NOV24-CDE'
exchange='coinbase'
trade_target='/'.join([base_asset,quote_asset]) if product_type.upper()=='SPOT' else  futures_target
trade_target

'DOGE/USDC'

In [10]:
live_env=BaseLiveTradingEnv(
            api_key=coinbase_api_key,
            api_secret=coinbase_api_secret,
            paper=test_net,
            symbol=trade_target,
            time_frame=time_frame,
            product_type=product_type,
            positions=[0,1],
            history_path='Trade_history/trade.db',
            exchange=exchange,
            forecast_model=forecast_model,
            discord_webhook='https://discord.com/api/webhooks/986694946381783102/FOA7nG9ShDcXY95-c3XEKV-Fdek66L9xfbQoKuEuFQkK2P4aFWaZ_fKmzw00j8Oj8Woj'

            )

Paper TradingFalse on coinbase at api.coinbase.com
 got symbol DOGE/USDC
final symbol DOGE-USDC
Connected to coinbase client
Getting data


100%|██████████| 78/78 [00:00<00:00, 12866.98it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [11]:
live_env.load_history()['date']

0           11-14-2024,
1           11-14-2024,
2           11-14-2024,
3           11-14-2024,
4           11-14-2024,
5           11-14-2024,
6           11-14-2024,
7           11-14-2024,
8           11-14-2024,
9           11-15-2024,
10          11-15-2024,
11    11-15-2024 04:00 
12    11-15-2024 05:00 
Name: date, dtype: object

In [12]:

agent_path='Agent/pearl_DOGEUSDT_Spot_model.pkl'
agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
                                        study_name='pearl-2024-11-12-hp-search',
                                        action_space_dim=2,
                                            observation_space_dim=30,)
agent=pearl_utils.load_agent_weights(agent,weight_path=agent_path)

[I 2024-11-15 10:12:36,687] Using an existing study with name 'pearl-2024-11-12-hp-search' instead of creating a new one.


In [13]:
from Pearl.pearl.utils.instantiations.environments.gym_environment import \
    GymEnvironment

In [14]:
live_pearl_env=GymEnvironment(live_env)

In [15]:
observation,action_space=live_pearl_env.reset()

Getting data


100%|██████████| 78/78 [00:00<00:00, 12277.39it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [16]:
agent.reset(observation, action_space)

In [17]:
action=agent.act(exploit=True)

# action=
action

tensor([0])

In [18]:
live_env.client.account()

,uuid,name,default,active,type,ready,hold,retail_portfolio_id,position_value,balance
currency,,,,,,,,,,
CORECHAIN,e03f1ea5-e129-564c-889b-194077bbf294,CORECHAIN Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'CORECHAIN'}",3f98d482-321e-54af-baa5-9b5927e52212,0,5.255967
NEAR,405446ef-08b1-5205-86e4-e505ab64ae5f,NEAR Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'NEAR'}",3f98d482-321e-54af-baa5-9b5927e52212,0,0.575780
SUI,a064f48a-1267-5b19-b0ff-d754d09c6f6b,SUI Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'SUI'}",3f98d482-321e-54af-baa5-9b5927e52212,0,66.104049
DOGE,a5630355-5534-5b9d-8b26-137d6e8992dd,DOGE Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'DOGE'}",3f98d482-321e-54af-baa5-9b5927e52212,0,0.033234
ETH2,12d0e91c-efa7-52b4-87f2-2953c6ea0180,ETH2 Wallet,True,True,ACCOUNT_TYPE_CRYPTO,False,"{'value': '0', 'currency': 'ETH2'}",3f98d482-321e-54af-baa5-9b5927e52212,0,0.000298
ATOM,99b5bef3-be97-5d57-b704-68532088d286,ATOM Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'ATOM'}",3f98d482-321e-54af-baa5-9b5927e52212,0,167.549281
USDC,c041014b-790b-5f17-8469-ced1f456940d,USDC Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'USDC'}",3f98d482-321e-54af-baa5-9b5927e52212,0,470.118677
USD,cfc0be7a-b031-500b-853f-963313341334,Cash (USD),False,True,ACCOUNT_TYPE_FIAT,True,"{'value': '0', 'currency': 'USD'}",3f98d482-321e-54af-baa5-9b5927e52212,0,647.830000
ETH,9834b834-02c5-50ca-bffa-2674c9c0650e,ETH Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'ETH'}",3f98d482-321e-54af-baa5-9b5927e52212,0,0.011981


In [19]:
action_result=live_pearl_env.step(int(action))


{'New_position': 0, 'Trade_from': 2.526503928964262e-05, 'Trade_to': 0, 'Change_size': -2.526503928964262e-05, 'Change_direction': -1, 'Place_order': 'SELL', 'Change needed to get to target': 2.526503928964262e-05, 'Size in dollars': 0.011877866779222389, 'Size in asset': 0.033254568506697994}
Sell
{'success': False, 'response': {'error': 'UNKNOWN_FAILURE_REASON', 'message': '', 'error_details': '', 'preview_failure_reason': 'PREVIEW_INVALID_BASE_SIZE_TOO_SMALL'}, 'order_configuration': {'market_market_ioc': {'base_size': '0.03', 'rfq_enabled': False, 'rfq_disabled': False}}}
ok...returning now....
fixed order: {'success': False, 'error': 'UNKNOWN_FAILURE_REASON', 'message': '', 'error_details': '', 'preview_failure_reason': 'PREVIEW_INVALID_BASE_SIZE_TOO_SMALL', 'fill_type': 'market_market_ioc', 'base_size': '0.03', 'rfq_enabled': False, 'rfq_disabled': False}
('portfolio_valuation', -1)
Getting data


100%|██████████| 78/78 [00:00<00:00, 11420.24it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

,New_position,Trade_from,Trade_to,Change_size,Change_direction,Place_order,Change needed to get to target,Size in dollars,Size in asset,success,...,data_symbol,position,time_frame,exchange,product_type,portfolio_distribution_asset,portfolio_distribution_fiat,portfolio_valuation,idx,real_position
0,0,0.000025,0,-0.000025,-1,SELL,0.000025,0.011878,0.033255,False,...,DOGE-USDC,0.000025,1h,coinbase,SPOT,0.033234,470.118677,470.130548,193,0.000025


In [20]:
len(live_env.client.trade_client.get_fills()['fills'])

33

In [21]:
action_result.observation

array([ 0.15894505,  0.1589216 ,  0.15889041,  0.15888518,  0.15888445,
        0.15888435,  0.15788688,  0.15783636,  0.15705043,  0.15703687,
        0.15710144,  0.15743642,  0.15941025,  0.15925704,  0.15916161,
        0.15917656,  0.15892012,  0.15885562,  0.1587474 ,  0.15867168,
        0.15836748,  0.1583662 ,  0.158302  ,  0.15839212,  0.00930297,
        0.07132278, -0.01426456,  0.1600366 ,  0.42793667,  0.42793667],
      dtype=float32)

In [22]:
df=live_env.df
import pandas as pd

In [23]:
df['ds_pst']=pd.to_datetime(df['ds'],utc=False)
df[['ds','ds_pst']]

,ds,ds_pst
1,2024-11-07 11:00:00,2024-11-07 11:00:00
2,2024-11-07 12:00:00,2024-11-07 12:00:00
3,2024-11-07 13:00:00,2024-11-07 13:00:00
4,2024-11-07 14:00:00,2024-11-07 14:00:00
5,2024-11-07 15:00:00,2024-11-07 15:00:00
...,...,...
190,2024-11-15 08:00:00,2024-11-15 08:00:00
191,2024-11-15 09:00:00,2024-11-15 09:00:00
192,2024-11-15 10:00:00,2024-11-15 10:00:00
193,2024-11-15 11:00:00,2024-11-15 11:00:00


In [24]:
live_env.client.get_trade_rules()

{'min_quote_size': 1.0,
 'max_quote_size': 10000000.0,
 'min_asset_size': 0.1,
 'max_asset_size': 141834300.0,
 'base_asset_precision': 1,
 'quote_asset_precision': 5}